# Progetto MCS

Per la gestione della struttura dati e le operazioni elementari fra matrici è richiesto di partire da una libreria open-source, come Eigen, Armadillo, blas/lapack. Oppure, qualora il linguaggio di programmazione lo permetta utilizzare vettori e matrici già implementate al suo interno.


## Data Import

In [2]:
import numpy as np
from scipy.io import mmread
from scipy.sparse import csr_matrix

In [3]:
from jacoby_mcs import Jacoby
ja = Jacoby()
ja.ciao()

hello world


In [ ]:
data = {
    "spa1" : 0,
    "spa2" : 0,
    "vem1" : 0,
    "vem2" : 0}
for x in data:
    data[x] = {
        "A" : mmread("data/" + x + ".mtx"),
        "x" : 0,
        "b" : 0,
    }
    data[x]["A"] = csr_matrix(data[x]["A"], (data[x]["A"]).shape).toarray()
    data[x]["x"] = [1] * data[x]["A"].shape[0]
    data[x]["b"] = data[x]["A"].dot(data[x]["x"])
    data[x]["x"] = np.random.rand(data[x]["A"].shape[1])

In [4]:
#import
spa1 = mmread('data/spa1.mtx')
spa2 = mmread('data/spa2.mtx')
vem1 = mmread('data/vem1.mtx')
vem2 = mmread('data/vem2.mtx')

In [5]:
spa1 = csr_matrix(spa1, (spa1).shape).toarray()
spa1

array([[9.61000e+02, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        5.00000e-02, 0.00000e+00],
       [0.00000e+00, 7.45000e+02, 1.43600e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 1.43600e+00, 5.24000e+02, ..., 0.00000e+00,
        0.00000e+00, 3.20000e-02],
       ...,
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.70548e+02,
        0.00000e+00, 0.00000e+00],
       [5.00000e-02, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        5.73000e+02, 0.00000e+00],
       [0.00000e+00, 0.00000e+00, 3.20000e-02, ..., 0.00000e+00,
        0.00000e+00, 1.49000e+02]])

In [18]:
spa2 = csr_matrix(spa2, (spa2).shape).toarray()
spa2

array([[2.8140e+03, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 7.8800e-01,
        0.0000e+00],
       [0.0000e+00, 1.9740e+03, 0.0000e+00, ..., 0.0000e+00, 9.6800e-01,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 1.4010e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 6.7512e+01, 0.0000e+00,
        0.0000e+00],
       [7.8800e-01, 9.6800e-01, 0.0000e+00, ..., 0.0000e+00, 2.1570e+03,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0230e+03]])

Creazione vettore $b$

In [6]:
spa1x = [1]*spa1.shape[0]
spa1b = spa1.dot(spa1x)
spa1b

array([1191.396,  938.016,  718.654,  552.596, 1189.838,  986.7  ,
        727.774, 1021.248,  633.1  ,  543.72 ,  758.848,  713.41 ,
        983.048,  574.32 , 1021.296, 1150.516, 1010.026,  334.834,
       1020.904,  288.246,  809.684,  229.748,  433.406,  369.084,
        413.068,  603.762, 1063.306, 1149.714,  255.51 ,  399.328,
        897.346,  215.106, 1176.538,  777.678,  896.398,  508.682,
        630.672,  802.308,  682.212, 1050.52 , 1142.068,  609.462,
        567.432,  889.814,  341.922,  724.162,  898.804,  472.34 ,
        207.12 , 1049.242,  577.666,  853.16 , 1087.68 , 1219.592,
        354.532,  637.78 ,  588.214,  208.514,  968.514, 1012.15 ,
        799.708,  663.904,  991.444, 1110.418,  261.62 , 1073.178,
        743.822,  982.466,  689.924,  202.292,  807.99 ,  641.784,
       1010.63 ,  532.502,  724.54 , 1150.992,  436.27 ,  726.23 ,
        432.766,  808.216,  932.454,  814.412,  780.946,  880.958,
        792.35 ,  910.964,  638.632,  550.854,  345.516,  411.

In [19]:
spa2x = [1]*spa2.shape[0]
spa2b = spa2.dot(spa2x)
spa2b

array([3385.904, 2596.528, 1995.15 , ...,  644.866, 2780.136, 1641.716])

Condizione di arresto: $\frac{||Ax^{(k)}-b||}{||b||}<tol$

Reminder: 

1.   **Iniziare le iterazioni con il vettore x nullo**
2.   **tol = [$10^{-4}, 10^{-6}, 10^{-8}, 10^{-10}]$**
3.   **Dichiarare di non essere giunti a convergenza se k > maxiter dove maxiter lo scegliamo (>= 20000)**




## Metodo di Jacobi
https://en.wikipedia.org/wiki/Jacobi_method

In [14]:
niter = 0
new_vector = np.asarray([0]*len(spa1x))
inverted_p_matrix = 1/np.diagonal(spa1)
tol = 10**(-4)
residual = spa1b - spa1.dot(new_vector)
while np.linalg.norm(residual)/np.linalg.norm(spa1b) >= tol and niter <= 20000:
    new_vector = new_vector + (inverted_p_matrix * (-residual))
    residual = spa1b - spa1.dot(new_vector)
    niter = niter +1
    print(np.linalg.norm(residual)/np.linalg.norm(spa1b))

2.7182540522775476
8.199894039642384
27.08024492462863
95.28450658422418
348.5385741731239
1303.8091952223526
4940.0660036686295
18856.54700298867
72292.36289106558
277895.7345887998
1070043.5949173633
4124738.489447309
15911497.96381261
61411176.46946063
237105625.433785
915697113.5145512
3537106239.3093457
13665010080.88482
52798604791.99958
204020840356.36255
788420507117.875
3046956452700.577
11775914223278.805
45513395000170.51
175912625233336.7
679932106607841.8
2628106196275396.5
1.015845184533017e+16
3.92661323079954e+16
1.51779693426982e+17
5.866962525786624e+17
2.2678606060622497e+18
8.766419023674473e+18
3.3886791068123034e+19
1.3099074822770565e+20
5.0635188800210194e+20
1.957337005558773e+21
7.566236605268751e+21
2.9247933038774733e+22
1.130605870341518e+23
4.370468122562998e+23
1.6894495741013066e+24
6.530748745823757e+24
2.524533320214197e+25
9.758870717438343e+25
3.772404855798455e+26
1.4582676933061452e+27
5.637108647544485e+27
2.1790927546997414e+28
8.423550087472575e

In [16]:
#metodo con calcolo singoli elemeti
iteration_vector = [0]*spa1.shape[0]
k = 0
tol = 10**(-4)
while (np.divide(np.linalg.norm((np.subtract(spa1.dot(iteration_vector), spa1b))), np.linalg.norm(spa1b)) >= tol) and k <= 20000:
  #vedere cosa del range e errori che da
  for i in range (0, spa1.shape[0]):
    sigma = 0
    for j in range(0, spa1.shape[0]):
      if(j!=i):
        sigma = sigma + spa1[i][j]*iteration_vector[j]
    iteration_vector[i] = (spa1b[i]-sigma)/spa1[i][i]
  k = k+1
  if(k%300 == 0):
    print(k)

KeyboardInterrupt: 

### Errore relativo - numero iterazioni - tempo di calcolo

In [21]:
#numero iterazioni
k

913

In [38]:
#errore relativo
np.linalg.norm(np.subtract(new_vector, spa1x))/np.linalg.norm(spa1x)

nan

## Metodo di Gauß-Seidel


## Metodo del Gradiente


## Metodo del Gradiente coniugato
